# Fooocus Colab

This scipt is not perfect but usable.. at least it woks on my machine 🤓

## Init

In [ ]:
import os
import glob
import json
import yaml
from google.colab import drive
from google.colab import files

config = {}
output_path = "/content/Fooocus/output"
civitai_api_key = ""

def mount_drive():
    mount_path = "/content/drive"
    if not os.path.exists(mount_path):
        drive.mount(mount_path)

def install_requerements():
    if not os.path.exists("/content/Fooocus"):
        !pip install -q civitdl pygit2==1.12.2
        %cd /content
        !git clone https://github.com/lllyasviel/Fooocus.git
        %cd /content/Fooocus
        !mv *.ogg notification.ogg

def write_to_file(config: str | dict | list, base_config_file: str, overwrite: bool = False):
    # Append file if file exists
    if os.path.isfile(base_config_file) and not overwrite:
        base_config = json.load(open(base_config_file, "r"))
        if isinstance(base_config, list) and isinstance(config, list):
            config = base_config + config
        elif isinstance(base_config, dict) and isinstance(config, dict):
            config = { **base_config, **config }
    # Write file
    out_file = open(base_config_file, "w")
    if isinstance(config, str):         # write string
        out_file.write(config)
    else:                               # write dict / list
        json.dump(config, out_file, indent=4)

def download(url: str, destination: str):
    global civitai_api_key
    if url.startswith("https://civitai.com/models/"):
        civitdl_params = f"{destination} --api-key {civitai_api_key} --no-model-overwrite"
        !civitdl {url} {civitdl_params}
    else:
        !wget --content-disposition -nc -P {destination} {url}

def batch_download(links: list, destination: str):
    if not isinstance(links, list) or len(links) == 0:
        return
    print(f"\n[Batch Download] {destination} of {len(links)} links")
    for key, item in links.items():
        print(f"Downloading : {key} to {destination}")
        download(item, destination)

def dict_print(obj: dict):
    print(json.dumps(obj, sort_keys=True, indent=4))

def run_ngrok_service(ngrok_token: str):
    !pip install pyngrok
    from pyngrok import ngrok, conf
    config = conf.PyngrokConfig(auth_token=ngrok_token)
    conn = ngrok.connect(7865, pyngrok_config=config, bind_tls=True)
    print(conn.public_url)

def copy_assets(from_dir: str, to_dir: str):
    print(f"Copy file {from_dir} to {to_dir}")
    !cp -r -n {from_dir} {to_dir}

def import_config():
    uploaded = files.upload()
    result = {}
    for file_name in uploaded.keys():
        try:
            result = yaml.safe_load(open(file_name))
        except yaml.YAMLError as exc:
            print(exc)
        !rm {file_name}
        return result

def generate_preset(preset: dict, file_name: str):
    preset_dir = "/content/Fooocus/presets/"
    if "default_model" not in preset:
        model_files = glob.glob(f"/content/Fooocus/models/checkpoints/**.safetensors")
        preset["default_model"] = model_files[0]
    print(f"[Generate Peset] {file_name}")
    write_to_file(preset, preset_dir + file_name)

def generate_style(style: dict, file_name: str):
    style_dir = "/content/Fooocus/sdxl_styles/"
    print(f"[Generate Style] {file_name} of {len(style)} Styles")
    write_to_file(style, style_dir + file_name, overwrite=True)

def generate_wildcard(wildcard: str, file_name: str):
    wildcard_dir = "/content/Fooocus/wildcards/"
    print(f"[Generate Wildcard] {file_name} of {len(str(wildcard).splitlines())} Lines")
    write_to_file(wildcard, wildcard_dir + file_name, overwrite=True)

def models_storage_stats():
    print("\n[Models Storage Stats]")
    !du -ms /content/Fooocus/models/checkpoints
    !du -ms /content/Fooocus/models/loras
    !du -ms /content/Fooocus/models/embeddings

def delete_unused_files(unused_files: list):
    for file in unused_files:
        if not os.path.exists(file):
            print(f"[Delete Unused] {file} : File Not Found")
            continue
        !rm {file}
        print(f"[Delete Unused] {file}")

## Main

In [ ]:
config = import_config()    # Input yaml config file
# dict_print(config)

gdrive = config["gdrive"]
civitai_api_key = config["civitai_api_key"]

install_requerements()
# run_ngrok_service(config["ngrok_token"])

if gdrive:
    output_path = gdrive.get("output_path", output_path)
    mount_drive()
    !mkdir -p {output_path}
    if gdrive.get("enable_restore_assets", False):
        for file in gdrive.get("backup_assets", []):
            copy_assets(
                from_dir=file["backup_dir"] + "/*",
                to_dir=file["working_dir"]
            )

for model in config.get("models", []):
    batch_download(model["models"], model["dir"])

for embedding in config.get("embeddings", []):
    batch_download(embedding["embeddings"], embedding["dir"])

for lora in config.get("loras", []):
    batch_download(lora["loras"], lora["dir"])

print() # newline
for preset in config.get("presets", []):
    generate_preset(preset["preset"], preset["file_name"])

print() # newline
for style in config.get("styles", []):
    generate_style(style["styles"], style["file_name"])

print() # newline
for wildcard in config.get("wildcards", []):
    if isinstance(wildcard["wildcard"], list):
        wildcard["wildcard"] = "\n".join(wildcard["wildcard"])
    generate_wildcard(wildcard["wildcard"], wildcard["file_name"])

if "unused" in config and isinstance(config["unused"], list):
    print() # newline
    delete_unused_files(config["unused"])


if gdrive and gdrive.get("enable_backup_assets", False):
    for file in gdrive.get("backup_assets", []):
        copy_assets(
            from_dir=file["working_dir"] + "/*",
            to_dir=file["backup_dir"]
        )

models_storage_stats()

preset = config["preset"]

In [ ]:
print("preset", preset)
print("output_path", output_path)

!python entry_with_update.py \
    --share \
    --always-high-vram \
    --preset {preset} \
    --output-path {output_path}